In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langgraph.graph import END, START, StateGraph, MessagesState
# from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode
from typing import Literal

In [ ]:
@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ["munich"]:
        return "It's 15 degrees Celsius and cloudy."
    else:
        return "It's 32 degrees Celsius and sunny."

In [ ]:
get_weather.invoke(input={"location": "munich"})

In [ ]:
tools = [get_weather]
model = ChatOllama(model="mistral", temperature=0).bind_tools(tools)

In [ ]:
model.invoke("hello")

In [ ]:
model.invoke("How is the weather in munich?")

In [ ]:
"""
MessagesState inherits a typedDict. 
also, it uses an AnnotatedMethod, to add the messages to the response itself.
"""
def call_model(state: MessagesState):
    messages = state["messages"]
    response = model.invoke(messages)
    return {"messages": [response]}

""" In Python, Literal is a type hint that allows you to specify that a variable can only take on specific, fixed values. It is part of the typing module and is used to indicate that a function parameter or return value must be one of a set of predefined values, rather than any value of a certain type. """
def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

In [ ]:
workflow = StateGraph(MessagesState)
tool_node = ToolNode(tools) 
""" The ToolNode class in the context of the LangGraph library is used to convert a list of tools into a node within a state graph. This allows the tools to be integrated into the workflow of the graph, enabling the model to invoke these tools as part of its processing. 

Functionality: When the graph reaches the ToolNode, it can invoke the associated tools based on the current state or messages, allowing for dynamic responses based on user input or other conditions.

"""

workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

In [ ]:
workflow.add_edge(START, "agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
)
workflow.add_edge("tools", "agent")

In [ ]:
graph = workflow.compile()

In [ ]:
from IPython.display import Image, display
from langchain_core.runnables.graph import MermaidDrawMethod

display(
    Image(
        graph.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API,
        )
    )
)

In [ ]:
from langchain_core.messages import HumanMessage

messages1 = [HumanMessage(content="Hello, how are you?")]
messages2 = [HumanMessage(content="How is the weather in munich?")]

In [ ]:
graph.invoke({"messages": messages1})

In [ ]:
graph.invoke({"messages": messages2})

In [ ]:
graph.invoke(
    {
        "messages": [
            HumanMessage(content="What would you recommend to do in that city than?")
        ]
    }
)

### Add Memory to chatbot

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

""" saves chat memory without using database """
checkpointer = MemorySaver()


In [ ]:
workflow = StateGraph(MessagesState)

workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
)
workflow.add_edge("tools", "agent")

graph = workflow.compile(checkpointer=checkpointer) # add the MemorySaver()

In [ ]:
graph.invoke(
    {"messages": [HumanMessage(content="How is the weather in munich?")]},
    config={"configurable": {"thread_id": 1}}, # access that chat memory using thread_id . 
)

In [ ]:
graph.invoke(
    {
        "messages": [
            HumanMessage(content="What would you recommend to do in that city than?")
        ]
    },
    config={"configurable": {"thread_id": 1}},
    # so in this chat, the memory from chat thread_id:1 is saved. so the llm knows the context of that.
)